In [3]:
import matplotlib.pyplot as plt
import sciris as sc
import numpy as np
import starsim as ss
from starsim.time import * # Import the date classes directly to make the examples more readable
Date = date

# V3 Time API

*Implementation goals*

- Want simulations to be safer - make it harder to write running code with incorrect results (e.g., forgetting a scaling factor, or having the units indirectly change)
- Balance type formality with flexibility, want to make things safe without being overly tedious (although lean towards explicit rather than implicit)
- Want simulations to be interoperable with calendar dates and numeric points in time (e.g., numerical years)

# Dates

In [ ]:
Date('2020-01-01')

In [ ]:
Date(2020)

In [ ]:
Date(2020.1)

## Arrays of dates

It is common to require an array of dates that is independent of any simulation timesteps - for example, interventions may have an attribute like `years = np.arange(2000,2020)` to align with data, even though the intervention is run with the same timestep as the main simulation. To facilitate these use cases, the `Date` class provides constructors for arrays as follows:

In [ ]:
Date.from_array([2000,2001,2002])

In [ ]:
Date.arange(2000,2003)

# Durations

In [ ]:
YearDur(1)

In [ ]:
YearDur(0.2)

In [ ]:
DateDur(days=1)

In [ ]:
DateDur(weeks=1)

In [ ]:
2*DateDur(weeks=1)

In [ ]:
DateDur(weeks=1)+DateDur(years=1)

This type only supports integers for each of the time periods. If division would result in a fractional number of units, they will be rolled over to smaller units. For example, half a week is 3.5 days, and the half-day is rolled over to become 12 hours:

In [ ]:
DateDur(weeks=1)/2

## Type interoperability

The two duration types are interoperable based on preset ratios

In [ ]:
Dur.ratios

For mixed types, the order determines the output type

In [ ]:
DateDur(weeks=1)+YearDur(1)

In [ ]:
YearDur(1)+DateDur(weeks=1)

In [ ]:
DateDur(weeks=1)+YearDur(1.5)

In [ ]:
DateDur(days=7)-DateDur(days=1)

In [ ]:
DateDur(weeks=1)-DateDur(days=1)

In [ ]:
(DateDur(days=7)-DateDur(days=1)).years

In [ ]:
(DateDur(weeks=1)-DateDur(days=1)).years

## Why have both duration types?

The two duration types are needed because simulations may or may not care about specific calendar dates during the year, and the variable duration of months and years (in terms of number of days) means that an exact mapping from one type of duration to the other is not possible. Essentially

- Some simulations will want to prioritise having a specific number of timesteps per year e.g., 12 or 52 timesteps per year
- Some simulations will want a specific number of weeks or months to elapse e.g., if the day of the week matters for things like school contacts

The example below demonstrates how 52 weeks with a `DateDur` leads to a different result compare to the approximation of 52 weeks per year, using a `YearDur` instance:


In [ ]:
Date('2020-01-01')+DateDur(days=1)

In [ ]:
Date('2020-01-01')+DateDur(weeks=1)

In [ ]:
Date('2020-01-01')+52*DateDur(weeks=1)

In [ ]:
Date('2020-01-01')+52*YearDur(1/52)

Subtraction of durations from dates is fine, and behaves as expected:

In [ ]:
Date('2020-01-01')-DateDur(days=1)

In [ ]:
Date('2020-01-01')-DateDur(days=6)

This example should be read as 'go back 1 week, then go forward 1 day' hence this is equivalent to going back 6 days, and gives the same result as the example above:

In [ ]:
Date('2020-01-01')-(DateDur(weeks=1)-DateDur(days=1))

## Combined constructor

The `Dur` constructor can be used as a single entry point for constructing `Dur` instances, where a numerical input leads to creation of a `YearDur` instance, while kwargs are used to produce a `DateDur` instance:

In [ ]:
Dur(1)

In [ ]:
Dur(years=1)

## Scaling factors

Quantities are absolute so scaling factors can be calculated on demand

In [ ]:
Dur(2)/Dur(1)

In [ ]:
Dur(weeks=1)/Dur(days=1)

In [ ]:
Dur(weeks=1)/Dur(1)

In [ ]:
Dur(weeks=1)/Dur(1/365)

In [ ]:
Dur(1/52)/Dur(1/365)

## Helper functions

To facilitate constructing common durations, helper functions are available:

In [ ]:
years(1)

In [ ]:
months(1)

In [ ]:
weeks(1)

In [ ]:
days(1)

# Numerical interoperability

A common use case is working with calendar dates, and therefore years as a natural unit of time. Although we would recommend using date and duration objects everywhere these quantities are used, it would make and migration easier for a significant proportion of users if bare numbers could be interpreted as years. This mainly applies to comparison operators, so for example:

In [ ]:
Date(2005)>2000

In [ ]:
YearDur(1) < 2

In [ ]:
DateDur(days=1)<2

This functionality is mainly aimed at easing the migration workload for users that have written code like `years = np.arange(2000,2020)`. However, we would recommend migration to `years = ss.Date.arange(2000,2020)` instead.

# Rates

Rates represent a value per unit time. They can be constructed either by taking the inverse of a duration, or by passing the value and associated period directly:

In short, we facilitate compatibility with `years = np.arange(2000,2020)` to decrease migration workload, but strongly recommend migration to `years = ss.Date.arange(2000,2020)`.

## Creating rates

In [ ]:
1/DateDur(days=1)

In [ ]:
Rate(1, Dur(1))

In [ ]:
Rate(1, Dur(months=1))

If no period is provided, 1 year is assumed:

In [ ]:
Rate(2)

## Scaling and adding rates

Rates can be scaled up or down multiplicatively. This operation does not affect the time period associated with the rate, so the action of scaling can be directly matched with the displayed value:

In [ ]:
2*Rate(2)

In [ ]:
4.5*Rate(1, Dur(months=1))

Rates can also be combined. When adding or subtracting rates, the reference time period will be drawn from the left hand side e.g.

In [ ]:
Rate(1,Dur(weeks=1))+Rate(1,Dur(days=1))

In [ ]:
Rate(1,Dur(days=1))+Rate(1,Dur(weeks=1))

This is intended to ease working with rates where the user has a natural reference time period (e.g., weeks, days) and wants to manipulate the rate using the same time period. So again, the action of the addition can be evaluated on the same time period as the original rate, with conversions of the other rate automatically applied as required. 

## Multiplication by durations

Crucially, rates can be multiplied by a duration to get the a dimensionless number per time period (usually per timestep). For example, if we have a rate of 1 per year, and multiply it by 2 years, we would get a value of 2

In [ ]:
Rate(1) * Dur(2)

Similarly, if we could have a rate of 1 per year, and multiply it by 1 week:

In [ ]:
Rate(1) * Dur(weeks=1)

In the example above, we two rates were added together, and the output rate depended on the order of the addition. However, these rates are equivalent and behave the same way (to within numerical precision) when multiplied by a duration e.g.,

In [ ]:
(Rate(1,Dur(weeks=1))+Rate(1,Dur(days=1)))*Dur(weeks=1) # <Rate: 8.0 per week> * 1 week = 8

In [ ]:
(Rate(1,Dur(days=1))+Rate(1,Dur(weeks=1)))*Dur(weeks=1) # <Rate: 1.142857...  per day> * 1 week = 8

## Division

Just as the inverse of a duration is a rate, the inverse of a rate is a duration:

In [ ]:
1/Rate(1,Dur(days=1))

If a rate is divided by another rate, the result is a dimensionless fraction e.g.

In [ ]:
Rate(14,weeks(1))/Rate(1,days(1))  # 14 per week, divided by 1 per day (=7 per week), gives 14/7=2

## Helper functions

To faciliate constructing rates, helper methods for common reference time periods are provided:

In [ ]:
peryear(1)

In [ ]:
permonth(1)

In [ ]:
perweek(1)

In [ ]:
perday(1)

# Probability rates

There are two other quantities closely related to rates

- A probability per unit time
- A rate that should be converted to a probability 

These are implemented with the `TimeProb` and `RateProb` classes, respectively. They are essentially identical to `Rate` instances, except that multiplication by a duration results in a probability being returned, which involves conversion to a cumulative hazard rate rather than just direct linear multiplication of the rate. For example, suppose we had a process where the probability per year was 0.1. This can be captured with:

In [ ]:
p = TimeProb(0.1, Dur(1))

If we multiply this by 1 year, we will get back the same probability

In [ ]:
p*Dur(1)

However, if we multiply it by 2 years to get the probability per 2 year period, we get less than double the value:

In [ ]:
p*Dur(2)

Similarly, if we multiply by half a year, to get the probability per 6 month period, we get more than half the value:

In [ ]:
p*Dur(0.5)

# Time vectors

Creation of time vectors is based around the `Date.arange` function but goes beyond that, because the `Time` object stored in Starsim modules has both date/dur and year-based representations, as well as explicitly tracks the timestep. After creating a `Time` object, it is necessary to initialise it. This is because initialisation accepts a `Sim` object, which is used to populate unspecified values based on the `Sim`. If no `Sim` is provided during initialisation, standard default values will be used. 

If all that is needed is an array of dates, use `Date.arange` e.g.

In [ ]:
Date.arange(Date('2020-01-01'), Date('2030-02-01'), Dur(days=1))

Alternatively, a full `Time` instance can be constructed, in which case `Time.tvec` corresponds to the dates:

In [ ]:
t = Time(Date('2020-01-01'), Date('2030-02-01'), Dur(days=1)).init()
t.tvec

In the same way that durations can be added to a single date, they can be added to an array of dates:

In [ ]:
t.tvec + Dur(days=1)

In [ ]:
t.tvec + Dur(1/12)

In [ ]:
t.tvec + Dur(months=1)

Plots can be made using dates on the x-axis e.g.,

In [ ]:
plt.plot(t.tvec, np.random.randn(len(t.tvec)));

Outbreak simulations would be run using a time vector based on durations

In [ ]:
t = Time(Dur(0), Dur(1), Dur(1/12)).init()
t.tvec

These can be plotted directly

In [ ]:
plt.plot(t.tvec, np.random.randn(len(t.tvec)));

In [ ]:
t = Time(Dur(days=0), Dur(days=30), Dur(days=1)).init()
t.tvec

These can be plotted directly as well - however, at the moment these are still in units of years elapsed - a todo item is to improve the default labelling (but at this stage does not appear to be worth delaying the release for). 

In [ ]:
plt.plot(t.tvec, np.random.randn(len(t.tvec)));

A date could then be added to these to align the outbreak with calendar dates for comparison to data

In [ ]:
t.tvec+Date('2020-01-01')

In [ ]:
t.tvec+Date('2020-03-01')

The yearvec is available as well

In [ ]:
t = Time(Date('2020-01-01'), Date('2030-02-01'), Dur(days=1)).init()
t.yearvec

In [ ]:
t = Time(Dur(0), Dur(1), Dur(1/12)).init()
t.yearvec

# Distributions

Distributions can be used without any changes, with dimensionless numbers interpreted as required by parent modules. In many cases, the distribution should be parametrized by a duration e.g., distribution of duration of infection. This is typically sampled per-agent, but for performance reasons, we need `Dist.rvs()` to return a bare numpy array that can be operated on natively. To achieve this, the distribution automatically converts any time parameters onto its parent timestep before calling `rvs`. Thus, the output of `rvs()` is specific to the timestep. Conversion is handled automatically. 

In order to demonstrate this functionality, it's necessary to also construct a mock module so that the `Dur` instance can carry out this conversion - normally of course, the `module` argument would correspond to the actual parent module for the distribution. In this case, we can see how a mean duration of 6 days maps to 6 timesteps, so the mean of the output from `d.rvs()` is 6:

In [ ]:
module = sc.objdict(t=sc.objdict(dt=Dur(days=1)))
d = ss.normal(Dur(days=6), Dur(days=1), module=module, strict=False)
d.init()
d.rvs(5)

Alternatively, consider exactly the same distribution, but in a case where the timestep is 1 week. Now, a mean duration of 6 days corresponds to a duration of 6/7 weeks. The output of `d.rvs()` now has a mean value of 6/7, even though the input parameters to `ss.normal` are unchanged. 

In [ ]:
module = sc.objdict(t=sc.objdict(dt=Dur(weeks=1)))
d = ss.normal(Dur(days=6), Dur(days=1), module=module, strict=False)
d.init()
d.rvs(5)

This conversion also works with function parameters, which can return time parameter instances. For instance


In [ ]:
def loc(module, sim, uids):
    return Dur(days=5)

module = sc.objdict(t=sc.objdict(dt=Dur(days=1)))
d = ss.normal(loc, Dur(days=1), module=module, strict=False)
d.init()
d.rvs(5)

In [ ]:
def loc(module, sim, uids):
    return np.array([Dur(days=x) for x in range(uids)])

module = sc.objdict(t=sc.objdict(dt=Dur(days=1)))
d = ss.normal(loc, Dur(days=1), module=module, strict=False)
d.init()
d.rvs(20)